# Stateful Syft Chat Tutorial

## Introduction

This tutorial demonstrates how to use the Stateful Syft Chat library, which adds persistent message storage using SQLite to the basic Syft Chat functionality. The stateful version ensures your message history is preserved across application restarts and provides more powerful filtering capabilities.

With Stateful Syft Chat, you can:
- Send and receive messages between Syft users
- Store message history in a SQLite database
- Retrieve message history even after restarting the application
- Filter messages by thread, time, or user with SQL-powered queries
- Track conversations with threading and replies
- Register custom message listeners for integration with your applications

### How It Works

The library combines client, server, and database components:
- **Server**: Uses EventRouter to process message requests
- **Database**: SQLite backend for message persistence
- **Client**: Sends messages to other users and processes responses

Messages are stored in a local SQLite database, allowing history to be preserved between sessions.

## 1. Setup

First, let's import the Stateful Syft Chat library and other dependencies:

In [1]:
import sys
sys.path.append('../..')
from examples.syft_chat_stateful import syft_chat
from datetime import datetime, timezone, timedelta
import os

### Environment Prerequisites

This notebook assumes you have set up Syft with at least one datasite. For a complete testing environment, you'll want to have two or more SyftBox clients running with different accounts.

For example:
- Alice's SyftBox on port 8082
- Bob's SyftBox on port 8081

You'll need to specify the correct config paths for your environment in the examples below.

### Database Configuration

Unlike the basic Syft Chat, the stateful version stores messages in a SQLite database. Let's specify the database paths for our clients:

In [2]:
# Define database paths for each client
bob_db_path = "bob_chat_messages.db"
alice_db_path = "alice_chat_messages.db"

# Remove existing databases if you want to start fresh
# Comment these out if you want to maintain history between notebook runs
if os.path.exists(bob_db_path):
    print(f"Removing existing database: {bob_db_path}")
    os.remove(bob_db_path)
    
if os.path.exists(alice_db_path):
    print(f"Removing existing database: {alice_db_path}")
    os.remove(alice_db_path)

Removing existing database: bob_chat_messages.db
Removing existing database: alice_chat_messages.db


## 2. Creating Chat Clients

Let's create clients connected to our SyftBox instances. Each client will start a chat server in the background and connect to its SQLite database.

In [3]:
# Create a client for Bob's account with its own database
bob_client = syft_chat.client("~/.syft_bob_config.json", db_path=bob_db_path)

2025-03-08 20:24:27.059 | INFO     | examples.syft_chat_stateful.syft_chat:__init__:256 - 🔑 Connected as: bob@openmined.org
2025-03-08 20:24:27.060 | INFO     | examples.syft_chat_stateful.syft_chat:_start_server:269 - 🔔 Server started for bob@openmined.org
2025-03-08 20:24:27.067 | INFO     | syft_event.server2:register_rpc:140 - Register RPC: /message
2025-03-08 20:24:27.067 | INFO     | syft_event.server2:register_rpc:140 - Register RPC: /history
2025-03-08 20:24:27.068 | INFO     | examples.syft_chat_stateful.syft_chat:_run_server:278 - 🚀 SERVER: Running syft_chat server as bob@openmined.org
2025-03-08 20:24:27.068 | INFO     | examples.syft_chat_stateful.syft_chat:_run_server:279 - 📡 SERVER: Listening for requests at /Users/atrask/Desktop/SyftBoxBob/datasites/bob@openmined.org/api_data/syft_chat/rpc
2025-03-08 20:24:27.072 | INFO     | syft_event.server2:publish_schema:96 - Published schema to /Users/atrask/Desktop/SyftBoxBob/datasites/bob@openmined.org/api_data/syft_chat/rpc/rpc.


🔔 NEW MESSAGE from alice@openmined.org: Hi Bob! I see your message, and it's now in our databases.


2025-03-08 20:27:06.276 | DEBUG    | syft_event.server2:process_pending_requests:105 - Processing pending request 89745628-6a94-41e1-ad40-676475884fd4.request
2025-03-08 20:27:06.277 | INFO     | examples.syft_chat_stateful.syft_chat:message_handler:89 - 📨 RECEIVED: Message from bob@openmined.org: Let's discuss Project Alpha in this thread. These ...



🔔 NEW MESSAGE from bob@openmined.org: Let's discuss Project Alpha in this thread. These thread messages are stored in the database.


2025-03-08 20:27:08.379 | DEBUG    | syft_event.server2:process_pending_requests:105 - Processing pending request 132a7e4a-36cf-4cac-b5b1-bcd9f08bf50e.request
2025-03-08 20:27:21.463 | DEBUG    | syft_event.server2:stop:123 - Stopping event loop


In [4]:
# Create a client for Alice's account with its own database
# Comment this out if you only have one Syft instance available
alice_client = syft_chat.client("~/.syft_alice_config.json", db_path=alice_db_path)

2025-03-08 20:24:27.780 | INFO     | examples.syft_chat_stateful.syft_chat:__init__:256 - 🔑 Connected as: alice@openmined.org
2025-03-08 20:24:27.781 | INFO     | examples.syft_chat_stateful.syft_chat:_start_server:269 - 🔔 Server started for alice@openmined.org
2025-03-08 20:24:27.787 | INFO     | syft_event.server2:register_rpc:140 - Register RPC: /message
2025-03-08 20:24:27.789 | INFO     | syft_event.server2:register_rpc:140 - Register RPC: /history
2025-03-08 20:24:27.790 | INFO     | examples.syft_chat_stateful.syft_chat:_run_server:278 - 🚀 SERVER: Running syft_chat server as alice@openmined.org
2025-03-08 20:24:27.791 | INFO     | examples.syft_chat_stateful.syft_chat:_run_server:279 - 📡 SERVER: Listening for requests at /Users/atrask/Desktop/SyftBoxAlice/datasites/alice@openmined.org/api_data/syft_chat/rpc
2025-03-08 20:24:27.799 | INFO     | syft_event.server2:publish_schema:96 - Published schema to /Users/atrask/Desktop/SyftBoxAlice/datasites/alice@openmined.org/api_data/syft

[01:27:40] FROM: alice@openmined.org (Thread: project-alpha)
MESSAGE: Here's a message that will trigger the advanced processor. This is stored in the database too.
--------------------------------------------------


## 3. Discovering Available Chat Users

Let's see which users have the chat service running:

In [5]:
# Get all available datasites
all_users = bob_client.list_all_users()
print(f"Total datasites: {len(all_users)}")

# Get only users with the chat service running
chat_users = bob_client.list_available_users()
print(f"Available chat users: {len(chat_users)}")
print("Chat-enabled users:")
for user in chat_users:
    print(f"  - {user}")

Total datasites: 140
Available chat users: 2
Chat-enabled users:
  - alice@openmined.org
  - bob@openmined.org


## 4. Sending and Receiving Messages

Now let's exchange some messages between users. Each message will be stored in the sender and recipient's databases.

### 4.1 Sending Messages

Let's send a message from Bob to Alice:

In [6]:
# If Alice's client is in the available users list, send a message to her
if 'alice@openmined.org' in chat_users:
    response = bob_client.send_message(
        'alice@openmined.org', 
        "Hello Alice! This message will be stored in both our databases."
    )
    print(f"Message sent with ID: {response.message_id}")
    print(f"Status: {response.status}")
else:
    # Find the first available user
    if chat_users:
        target = chat_users[0]
        print(f"Alice not found, messaging {target} instead")
        response = bob_client.send_message(target, "Hello! This is a test message.")
        print(f"Message sent with ID: {response.message_id}")
    else:
        print("No chat users found to message")

2025-03-08 20:24:31.529 | INFO     | examples.syft_chat_stateful.syft_chat:send_message:339 - 📤 SENDING: Message to alice@openmined.org
2025-03-08 20:24:35.233 | DEBUG    | syft_event.handlers:on_any_event:31 - FSEvent - created - /Users/atrask/Desktop/SyftBoxAlice/datasites/alice@openmined.org/api_data/syft_chat/rpc/message/40709db2-e7d0-42ee-9129-08912c52c53d.request
2025-03-08 20:24:35.234 | INFO     | examples.syft_chat_stateful.syft_chat:message_handler:89 - 📨 RECEIVED: Message from bob@openmined.org: Hello Alice! This message will be stored in both o...
2025-03-08 20:24:39.343 | INFO     | examples.syft_chat_stateful.syft_chat:send_message:356 - 📥 RECEIVED: Delivery confirmation from alice@openmined.org. Time: 7.81s
2025-03-08 20:24:39.354 | DEBUG    | syft_event.handlers:on_any_event:31 - FSEvent - created - /Users/atrask/Desktop/SyftBoxBob/datasites/bob@openmined.org/api_data/syft_chat/rpc/message/dde39722-4878-4b0a-8c5b-c29d6793d763.request
2025-03-08 20:24:39.355 | INFO     |

Message sent with ID: 9f216081-792d-4063-8cf9-7c1a669efd71
Status: delivered


### 4.2 Receiving Messages with Listeners

To actively process incoming messages, we can register message listeners. These are functions that will be called whenever a new message is received.

In [7]:
# Define a custom message listener function
def print_message(message):
    print(f"\n🔔 NEW MESSAGE from {message.sender}: {message.content}")

# Register the listener with Bob's client
bob_client.add_message_listener(print_message)

# Also register with Alice's client if available
try:
    alice_client.add_message_listener(print_message)
except NameError:
    pass

### 4.3 Bidirectional Chat

If we have both Alice and Bob's clients running, we can demonstrate bidirectional chat:

In [8]:
# Check if we have Alice's client available
try:
    # Alice responds to Bob
    alice_response = alice_client.send_message(
        'bob@openmined.org',
        "Hi Bob! I see your message, and it's now in our databases."
    )
    print("Message sent from Alice to Bob.")
except NameError:
    print("Alice client not available. Cannot demonstrate bidirectional chat.")

2025-03-08 20:24:43.645 | INFO     | examples.syft_chat_stateful.syft_chat:send_message:339 - 📤 SENDING: Message to bob@openmined.org
2025-03-08 20:24:47.553 | DEBUG    | syft_event.handlers:on_any_event:31 - FSEvent - created - /Users/atrask/Desktop/SyftBoxBob/datasites/bob@openmined.org/api_data/syft_chat/rpc/message/b5c6d41e-3a1e-499f-9903-362dd5ade7af.request
2025-03-08 20:24:47.554 | INFO     | examples.syft_chat_stateful.syft_chat:message_handler:89 - 📨 RECEIVED: Message from alice@openmined.org: Hi Bob! I see your message, and it's now in our da...



🔔 NEW MESSAGE from alice@openmined.org: Hi Bob! I see your message, and it's now in our databases.


2025-03-08 20:24:51.894 | INFO     | examples.syft_chat_stateful.syft_chat:send_message:356 - 📥 RECEIVED: Delivery confirmation from bob@openmined.org. Time: 8.25s
2025-03-08 20:24:51.905 | DEBUG    | syft_event.handlers:on_any_event:31 - FSEvent - created - /Users/atrask/Desktop/SyftBoxAlice/datasites/alice@openmined.org/api_data/syft_chat/rpc/message/c9064466-a732-4428-8e64-b101099a5c9a.request
2025-03-08 20:24:51.906 | INFO     | examples.syft_chat_stateful.syft_chat:message_handler:89 - 📨 RECEIVED: Message from alice@openmined.org: Hi Bob! I see your message, and it's now in our da...



🔔 NEW MESSAGE from alice@openmined.org: Hi Bob! I see your message, and it's now in our databases.
Message sent from Alice to Bob.


## 5. Working with Chat History

One of the biggest advantages of the stateful version is its ability to retrieve message history from the database. Let's explore these features.

### 5.1 Local Chat History

Each client can retrieve messages from its local SQLite database:

In [9]:
# Get Bob's chat history from the database
bob_history = bob_client.get_chat_history()
print(f"Bob's message history ({len(bob_history)} messages):")
for msg in bob_history:
    time_str = msg.timestamp.strftime("%Y-%m-%d %H:%M:%S %Z")
    print(f"{time_str} - {msg.sender}: {msg.content}")

2025-03-08 20:25:08.429 | DEBUG    | syft_event.handlers:on_any_event:31 - FSEvent - created - /Users/atrask/Desktop/SyftBoxBob/datasites/bob@openmined.org/api_data/syft_chat/rpc/history/132a7e4a-36cf-4cac-b5b1-bcd9f08bf50e.request


Bob's message history (2 messages):
2025-03-09 01:24:31  - bob@openmined.org: Hello Alice! This message will be stored in both our databases.
2025-03-09 01:24:43  - alice@openmined.org: Hi Bob! I see your message, and it's now in our databases.


In [10]:
# We can filter history by sender (now using SQL filtering)
bob_alice_msgs = bob_client.get_chat_history(with_user='alice@openmined.org')
print(f"Bob's filtered chat history ({len(bob_alice_msgs)} messages):")
for msg in bob_alice_msgs:
    time_str = msg.timestamp.strftime("%Y-%m-%d %H:%M:%S %Z")
    print(f"{time_str} - {msg.sender}: {msg.content}")

2025-03-08 20:25:09.952 | DEBUG    | syft_event.handlers:on_any_event:31 - FSEvent - created - /Users/atrask/Desktop/SyftBoxBob/datasites/bob@openmined.org/api_data/syft_chat/rpc/history/b7c183c0-8d8c-4c0a-9e6b-dc5d8f861242.request


Bob's filtered chat history (2 messages):
2025-03-09 01:24:31  - bob@openmined.org: Hello Alice! This message will be stored in both our databases.
2025-03-09 01:24:43  - alice@openmined.org: Hi Bob! I see your message, and it's now in our databases.


### 5.2 Remote Chat History

You can also request chat history from another user. The stateful version will store copies of these messages in your local database:

In [11]:
# Request history from Alice's client
if 'alice@openmined.org' in chat_users:
    alice_history = bob_client.request_history_from_user('alice@openmined.org')
    print(f"Retrieved {len(alice_history)} messages from Alice's history:")
    for msg in alice_history:
        time_str = msg.timestamp.strftime("%Y-%m-%d %H:%M:%S %Z")
        print(f"{time_str} - {msg.sender}: {msg.content}")
else:
    print("Alice not available to request history from.")

2025-03-08 20:25:11.517 | INFO     | examples.syft_chat_stateful.syft_chat:request_history_from_user:432 - 📤 REQUESTING: Chat history from alice@openmined.org
2025-03-08 20:25:14.080 | DEBUG    | syft_event.handlers:on_any_event:31 - FSEvent - created - /Users/atrask/Desktop/SyftBoxAlice/datasites/alice@openmined.org/api_data/syft_chat/rpc/history/7c731730-48a2-43f3-b062-387a348c5b5e.request
2025-03-08 20:25:18.055 | INFO     | examples.syft_chat_stateful.syft_chat:request_history_from_user:448 - 📥 RECEIVED: History from alice@openmined.org (3 messages). Time: 6.54s
2025-03-08 20:25:18.135 | DEBUG    | syft_event.handlers:on_any_event:31 - FSEvent - created - /Users/atrask/Desktop/SyftBoxBob/datasites/bob@openmined.org/api_data/syft_chat/rpc/message/cd50b5ff-af96-4169-bb54-1840bdf7a220.request
2025-03-08 20:25:18.136 | INFO     | examples.syft_chat_stateful.syft_chat:message_handler:89 - 📨 RECEIVED: Message from alice@openmined.org: Hi Bob! I see your message, and it's now in our da...


🔔 NEW MESSAGE from alice@openmined.org: Hi Bob! I see your message, and it's now in our databases.

🔔 NEW MESSAGE from bob@openmined.org: Hello Alice! This message will be stored in both our databases.

🔔 NEW MESSAGE from alice@openmined.org: Hi Bob! I see your message, and it's now in our databases.
Retrieved 3 messages from Alice's history:
2025-03-09 01:22:59  - alice@openmined.org: Hi Bob! I see your message, and it's now in our databases.
2025-03-09 01:24:31  - bob@openmined.org: Hello Alice! This message will be stored in both our databases.
2025-03-09 01:24:43  - alice@openmined.org: Hi Bob! I see your message, and it's now in our databases.


## 6. Advanced Features

The stateful version supports the same advanced features as the basic version but with better persistence.

### 6.1 Threaded Conversations

You can group messages into threads which are stored in the database:

In [12]:
# Create a new thread
thread_id = "project-alpha"

try:
    # Send a message in this thread
    response = bob_client.send_message(
        'alice@openmined.org',
        "Let's discuss Project Alpha in this thread. These thread messages are stored in the database.",
        thread_id=thread_id
    )
    print(f"Started a new thread with ID: {thread_id}")
except Exception as e:
    print(f"Error: {e}")

2025-03-08 20:26:59.625 | INFO     | examples.syft_chat_stateful.syft_chat:send_message:339 - 📤 SENDING: Message to alice@openmined.org
2025-03-08 20:27:02.365 | DEBUG    | syft_event.handlers:on_any_event:31 - FSEvent - created - /Users/atrask/Desktop/SyftBoxAlice/datasites/alice@openmined.org/api_data/syft_chat/rpc/message/dc800bdb-0f49-4c75-81a1-87cc78e41226.request
2025-03-08 20:27:02.366 | INFO     | examples.syft_chat_stateful.syft_chat:message_handler:89 - 📨 RECEIVED: Message from bob@openmined.org: Let's discuss Project Alpha in this thread. These ...



🔔 NEW MESSAGE from bob@openmined.org: Let's discuss Project Alpha in this thread. These thread messages are stored in the database.


2025-03-08 20:27:06.268 | INFO     | examples.syft_chat_stateful.syft_chat:send_message:356 - 📥 RECEIVED: Delivery confirmation from alice@openmined.org. Time: 6.64s
2025-03-08 20:27:06.289 | DEBUG    | syft_event.handlers:on_any_event:31 - FSEvent - created - /Users/atrask/Desktop/SyftBoxBob/datasites/bob@openmined.org/api_data/syft_chat/rpc/message/89745628-6a94-41e1-ad40-676475884fd4.request
2025-03-08 20:27:06.290 | INFO     | examples.syft_chat_stateful.syft_chat:message_handler:89 - 📨 RECEIVED: Message from bob@openmined.org: Let's discuss Project Alpha in this thread. These ...
2025-03-08 20:27:06.290 | INFO     | examples.syft_chat_stateful.syft_chat:message_handler:119 - Message with ID 986320e6-dfef-4d5a-886f-a458681c7c88 already exists, skipping database insert



🔔 NEW MESSAGE from bob@openmined.org: Let's discuss Project Alpha in this thread. These thread messages are stored in the database.
Started a new thread with ID: project-alpha


### 6.2 Message Replies

You can reply to specific messages using the `reply_to` parameter:

In [13]:
try:
    # Get Bob's last message ID
    thread_messages = bob_client.get_chat_history()
    last_msg = [m for m in thread_messages if m.thread_id == thread_id][0]
    
    # Alice replies to Bob's message
    alice_response = alice_client.send_message(
        'bob@openmined.org',
        "I'm ready to discuss Project Alpha. What aspects should we focus on first? This reply is linked in the database.",
        thread_id=thread_id,
        reply_to=last_msg.msg_id
    )
    print("Alice replied to the thread message")
except Exception as e:
    print(f"Error: {e}")

2025-03-08 20:27:08.450 | DEBUG    | syft_event.handlers:on_any_event:31 - FSEvent - created - /Users/atrask/Desktop/SyftBoxBob/datasites/bob@openmined.org/api_data/syft_chat/rpc/history/132a7e4a-36cf-4cac-b5b1-bcd9f08bf50e.request
2025-03-08 20:27:08.483 | INFO     | examples.syft_chat_stateful.syft_chat:send_message:339 - 📤 SENDING: Message to bob@openmined.org
2025-03-08 20:27:12.701 | DEBUG    | syft_event.handlers:on_any_event:31 - FSEvent - created - /Users/atrask/Desktop/SyftBoxBob/datasites/bob@openmined.org/api_data/syft_chat/rpc/message/d1bd19df-0940-4197-a9a7-e1c87bcc6d6f.request
2025-03-08 20:27:12.702 | INFO     | examples.syft_chat_stateful.syft_chat:message_handler:89 - 📨 RECEIVED: Message from alice@openmined.org: I'm ready to discuss Project Alpha. What aspects s...



🔔 NEW MESSAGE from alice@openmined.org: I'm ready to discuss Project Alpha. What aspects should we focus on first? This reply is linked in the database.


2025-03-08 20:27:16.921 | INFO     | examples.syft_chat_stateful.syft_chat:send_message:356 - 📥 RECEIVED: Delivery confirmation from bob@openmined.org. Time: 8.44s
2025-03-08 20:27:16.932 | DEBUG    | syft_event.handlers:on_any_event:31 - FSEvent - created - /Users/atrask/Desktop/SyftBoxAlice/datasites/alice@openmined.org/api_data/syft_chat/rpc/message/b3369338-2c42-4996-a311-ab84eacc95d0.request
2025-03-08 20:27:16.933 | INFO     | examples.syft_chat_stateful.syft_chat:message_handler:89 - 📨 RECEIVED: Message from alice@openmined.org: I'm ready to discuss Project Alpha. What aspects s...



🔔 NEW MESSAGE from alice@openmined.org: I'm ready to discuss Project Alpha. What aspects should we focus on first? This reply is linked in the database.
Alice replied to the thread message


### 6.3 Time Filters

You can filter chat history by time using SQL-powered queries:

In [14]:
# Get messages from the last hour using SQL filtering
one_hour_ago = datetime.now(timezone.utc) - timedelta(hours=1)
recent_messages = bob_client.get_chat_history(since=one_hour_ago)

print(f"Messages in the last hour: {len(recent_messages)}")
for msg in recent_messages:
    time_str = msg.timestamp.strftime("%Y-%m-%d %H:%M:%S %Z")
    print(f"{time_str} - {msg.sender}: {msg.content}")

2025-03-08 20:27:18.366 | DEBUG    | syft_event.handlers:on_any_event:31 - FSEvent - created - /Users/atrask/Desktop/SyftBoxBob/datasites/bob@openmined.org/api_data/syft_chat/rpc/history/f6bd2da4-1a8e-45e9-a4a7-e7204f1f41c5.request


Messages in the last hour: 5
2025-03-09 01:22:59  - alice@openmined.org: Hi Bob! I see your message, and it's now in our databases.
2025-03-09 01:24:31  - bob@openmined.org: Hello Alice! This message will be stored in both our databases.
2025-03-09 01:24:43  - alice@openmined.org: Hi Bob! I see your message, and it's now in our databases.
2025-03-09 01:26:59  - bob@openmined.org: Let's discuss Project Alpha in this thread. These thread messages are stored in the database.
2025-03-09 01:27:08  - alice@openmined.org: I'm ready to discuss Project Alpha. What aspects should we focus on first? This reply is linked in the database.


## 7. Demonstrating Statefulness

The key advantage of the stateful version is that message history persists even when clients are restarted. Let's demonstrate this feature:

### 7.1 Shutting Down and Restarting Clients

First, we'll shut down our clients and then restart them:

In [15]:
# Close both clients
print("Shutting down Bob's client...")
bob_client.close()

try:
    print("Shutting down Alice's client...")
    alice_client.close()
except NameError:
    pass

print("All clients shut down")

2025-03-08 20:27:21.413 | INFO     | examples.syft_chat_stateful.syft_chat:close:515 - 👋 Shutting down syft_chat client...
2025-03-08 20:27:21.468 | INFO     | examples.syft_chat_stateful.syft_chat:close:515 - 👋 Shutting down syft_chat client...


Shutting down Bob's client...
Shutting down Alice's client...
All clients shut down


In [16]:
# Restart the clients (connecting to the existing databases)
print("Restarting Bob's client...")
bob_client = syft_chat.client("~/.syft_bob_config.json", db_path=bob_db_path)

try:
    print("Restarting Alice's client...")
    alice_client = syft_chat.client("~/.syft_alice_config.json", db_path=alice_db_path)
except Exception as e:
    print(f"Could not restart Alice's client: {e}")
    
print("Clients restarted successfully")

2025-03-08 20:27:22.650 | INFO     | examples.syft_chat_stateful.syft_chat:__init__:256 - 🔑 Connected as: bob@openmined.org
2025-03-08 20:27:22.651 | INFO     | examples.syft_chat_stateful.syft_chat:_start_server:269 - 🔔 Server started for bob@openmined.org
2025-03-08 20:27:22.652 | INFO     | examples.syft_chat_stateful.syft_chat:__init__:256 - 🔑 Connected as: alice@openmined.org
2025-03-08 20:27:22.652 | INFO     | syft_event.server2:register_rpc:140 - Register RPC: /message
2025-03-08 20:27:22.652 | INFO     | examples.syft_chat_stateful.syft_chat:_start_server:269 - 🔔 Server started for alice@openmined.org


Restarting Bob's client...
Restarting Alice's client...
Clients restarted successfully


2025-03-08 20:27:22.654 | INFO     | syft_event.server2:register_rpc:140 - Register RPC: /history
2025-03-08 20:27:22.654 | INFO     | examples.syft_chat_stateful.syft_chat:_run_server:278 - 🚀 SERVER: Running syft_chat server as bob@openmined.org
2025-03-08 20:27:22.655 | INFO     | examples.syft_chat_stateful.syft_chat:_run_server:279 - 📡 SERVER: Listening for requests at /Users/atrask/Desktop/SyftBoxBob/datasites/bob@openmined.org/api_data/syft_chat/rpc
2025-03-08 20:27:22.655 | INFO     | syft_event.server2:register_rpc:140 - Register RPC: /message
2025-03-08 20:27:22.658 | INFO     | syft_event.server2:register_rpc:140 - Register RPC: /history
2025-03-08 20:27:22.658 | INFO     | examples.syft_chat_stateful.syft_chat:_run_server:278 - 🚀 SERVER: Running syft_chat server as alice@openmined.org
2025-03-08 20:27:22.659 | INFO     | examples.syft_chat_stateful.syft_chat:_run_server:279 - 📡 SERVER: Listening for requests at /Users/atrask/Desktop/SyftBoxAlice/datasites/alice@openmined.org

[01:27:40] FROM: alice@openmined.org (Thread: project-alpha)
MESSAGE: Here's a message that will trigger the advanced processor. This is stored in the database too.
--------------------------------------------------


2025-03-08 20:27:50.569 | DEBUG    | syft_event.server2:stop:123 - Stopping event loop
2025-03-08 20:27:50.628 | DEBUG    | syft_event.server2:stop:123 - Stopping event loop


### 7.2 Verifying Persistence

Now, let's verify that our message history is still available after restart:

In [17]:
# Check if our message history is still available
bob_history_after_restart = bob_client.get_chat_history()
print(f"Bob's message history after restart ({len(bob_history_after_restart)} messages):")
for msg in bob_history_after_restart:
    time_str = msg.timestamp.strftime("%Y-%m-%d %H:%M:%S %Z")
    print(f"{time_str} - {msg.sender}: {msg.content}")

Bob's message history after restart (5 messages):
2025-03-09 01:22:59  - alice@openmined.org: Hi Bob! I see your message, and it's now in our databases.
2025-03-09 01:24:31  - bob@openmined.org: Hello Alice! This message will be stored in both our databases.
2025-03-09 01:24:43  - alice@openmined.org: Hi Bob! I see your message, and it's now in our databases.
2025-03-09 01:26:59  - bob@openmined.org: Let's discuss Project Alpha in this thread. These thread messages are stored in the database.
2025-03-09 01:27:08  - alice@openmined.org: I'm ready to discuss Project Alpha. What aspects should we focus on first? This reply is linked in the database.


### 7.3 Thread Persistence

Let's also verify that our thread information is preserved:

In [18]:
# Check if thread information is preserved
thread_messages = bob_client.get_chat_history()
thread_messages = [m for m in thread_messages if m.thread_id == thread_id]

print(f"Messages in the '{thread_id}' thread after restart ({len(thread_messages)} messages):")
for msg in thread_messages:
    time_str = msg.timestamp.strftime("%Y-%m-%d %H:%M:%S %Z")
    reply_info = f" (Reply to: {msg.reply_to})" if msg.reply_to else ""
    print(f"{time_str} - {msg.sender}{reply_info}: {msg.content}")

Messages in the 'project-alpha' thread after restart (2 messages):
2025-03-09 01:26:59  - bob@openmined.org: Let's discuss Project Alpha in this thread. These thread messages are stored in the database.
2025-03-09 01:27:08  - alice@openmined.org (Reply to: 986320e6-dfef-4d5a-886f-a458681c7c88): I'm ready to discuss Project Alpha. What aspects should we focus on first? This reply is linked in the database.


### 7.4 Continuing Conversations After Restart

Now, let's continue our conversation after restart to show that everything still works:

In [19]:
# Send a new message after restart
try:
    after_restart_msg = bob_client.send_message(
        'alice@openmined.org',
        "This message was sent after restarting the clients. Our conversation history is persistent!",
        thread_id=thread_id
    )
    print("Message sent after restart")
except Exception as e:
    print(f"Error: {e}")

2025-03-08 20:27:28.733 | INFO     | examples.syft_chat_stateful.syft_chat:send_message:339 - 📤 SENDING: Message to alice@openmined.org
2025-03-08 20:27:31.267 | DEBUG    | syft_event.handlers:on_any_event:31 - FSEvent - created - /Users/atrask/Desktop/SyftBoxAlice/datasites/alice@openmined.org/api_data/syft_chat/rpc/message/eb5141d8-93dc-43d6-8735-785555223773.request
2025-03-08 20:27:31.268 | INFO     | examples.syft_chat_stateful.syft_chat:message_handler:89 - 📨 RECEIVED: Message from bob@openmined.org: This message was sent after restarting the clients...
2025-03-08 20:27:31.269 | INFO     | examples.syft_chat_stateful.syft_chat:message_handler:119 - Message with ID 6f2151fd-3084-4ef7-8f14-c98b8abdb8cd already exists, skipping database insert
2025-03-08 20:27:35.662 | INFO     | examples.syft_chat_stateful.syft_chat:send_message:356 - 📥 RECEIVED: Delivery confirmation from alice@openmined.org. Time: 6.93s


Message sent after restart


### 7.5 Examining the Database

For the curious, we can directly examine the SQLite database to see the stored messages:

In [20]:
# This is optional - requires sqlite3 module
import sqlite3

# Connect to Bob's database
conn = sqlite3.connect(bob_db_path)
cursor = conn.cursor()

# Query all messages
cursor.execute("SELECT msg_id, sender, content, thread_id, reply_to FROM messages")
rows = cursor.fetchall()

print(f"Direct database query found {len(rows)} messages:")
for row in rows:
    msg_id, sender, content, thread_id, reply_to = row
    thread_info = f"(Thread: {thread_id})" if thread_id else ""
    reply_info = f"(Reply to: {reply_to})" if reply_to else ""
    print(f"{sender} {thread_info} {reply_info}: {content[:50]}...")
    
conn.close()

Direct database query found 6 messages:
bob@openmined.org  : Hello Alice! This message will be stored in both o...
alice@openmined.org  : Hi Bob! I see your message, and it's now in our da...
alice@openmined.org  : Hi Bob! I see your message, and it's now in our da...
bob@openmined.org (Thread: project-alpha) : Let's discuss Project Alpha in this thread. These ...
alice@openmined.org (Thread: project-alpha) (Reply to: 986320e6-dfef-4d5a-886f-a458681c7c88): I'm ready to discuss Project Alpha. What aspects s...
bob@openmined.org (Thread: project-alpha) : This message was sent after restarting the clients...


## 8. Custom Message Processing

You can create message listeners that can access the database for advanced processing:

In [21]:
# A more advanced message processor
def advanced_message_processor(message):
    # Check if this is part of a thread
    thread_info = f" (Thread: {message.thread_id})" if message.thread_id else ""
    reply_info = f" (Reply to: {message.reply_to})" if message.reply_to else ""
    
    # Format the message nicely
    time_str = message.timestamp.strftime("%H:%M:%S")
    print(f"[{time_str}] FROM: {message.sender}{thread_info}{reply_info}")
    print(f"MESSAGE: {message.content}")
    print("-" * 50)

# Replace the existing listener with our advanced one
try:
    bob_client.remove_message_listener(print_message)
    bob_client.add_message_listener(advanced_message_processor)
    print("Registered advanced message processor")
except Exception as e:
    print(f"Error setting up listener: {e}")

Registered advanced message processor


In [22]:
# Test the advanced message processor
try:
    # Alice sends another message
    alice_response = alice_client.send_message(
        'bob@openmined.org',
        "Here's a message that will trigger the advanced processor. This is stored in the database too.",
        thread_id=thread_id
    )
    print("Advanced processing works!")
except Exception as e:
    print(f"Error: {e}")

2025-03-08 20:27:40.074 | INFO     | examples.syft_chat_stateful.syft_chat:send_message:339 - 📤 SENDING: Message to bob@openmined.org
2025-03-08 20:27:48.086 | INFO     | examples.syft_chat_stateful.syft_chat:send_message:356 - 📥 RECEIVED: Delivery confirmation from bob@openmined.org. Time: 8.01s
2025-03-08 20:27:48.097 | DEBUG    | syft_event.handlers:on_any_event:31 - FSEvent - created - /Users/atrask/Desktop/SyftBoxAlice/datasites/alice@openmined.org/api_data/syft_chat/rpc/message/bf8042a2-9c33-4179-b9a8-d89775810838.request
2025-03-08 20:27:48.098 | INFO     | examples.syft_chat_stateful.syft_chat:message_handler:89 - 📨 RECEIVED: Message from alice@openmined.org: Here's a message that will trigger the advanced pr...


Advanced processing works!


## 9. Cleanup

When we're done, it's important to properly close all clients to shut down the background servers:

In [23]:
# Close all clients
bob_client.close()
try:
    alice_client.close()
except NameError:
    pass

print("All clients closed")

2025-03-08 20:27:50.466 | INFO     | examples.syft_chat_stateful.syft_chat:close:515 - 👋 Shutting down syft_chat client...
2025-03-08 20:27:50.573 | INFO     | examples.syft_chat_stateful.syft_chat:close:515 - 👋 Shutting down syft_chat client...


All clients closed


## 10. Conclusion

In this tutorial, we've demonstrated the Stateful Syft Chat library, which adds SQLite persistence to the basic chat functionality. The key advantages include:

1. **Persistent Storage** - Messages are stored in a SQLite database
2. **Session Independence** - Message history is preserved across restarts
3. **SQL-Powered Filtering** - More powerful query capabilities for history
4. **Data Consistency** - Message records are maintained reliably
5. **Scalability** - Better handling of larger message histories

The stateful version maintains all the features of the basic version while adding these important capabilities.

### Next Steps

- Build applications on top of the stateful chat system
- Implement additional database features like message archiving
- Create data analysis tools for chat history
- Extend the database schema with additional metadata fields
- Implement encrypted storage for sensitive messages